In [6]:
import boto3
import uuid

# Initialize DynamoDB client (configure your AWS credentials before running this)
dynamodb = boto3.resource("dynamodb")  # e.g., 'us-east-1'


# --------- Create schedule_task_table ---------
def create_schedule_task_table():
    table = dynamodb.create_table(
        TableName="schedule_task_table",
        KeySchema=[{"AttributeName": "task_id", "KeyType": "HASH"}],
        AttributeDefinitions=[{"AttributeName": "task_id", "AttributeType": "S"}],
        BillingMode="PAY_PER_REQUEST",
    )
    table.wait_until_exists()
    print("Created table: schedule_task_table")


# --------- Create schedule_subtask_table ---------
def create_schedule_subtask_table():
    table = dynamodb.create_table(
        TableName="schedule_subtask_table",
        KeySchema=[{"AttributeName": "subtask_id", "KeyType": "HASH"}],
        AttributeDefinitions=[{"AttributeName": "subtask_id", "AttributeType": "S"}],
        BillingMode="PAY_PER_REQUEST",
    )
    table.wait_until_exists()
    print("Created table: schedule_subtask_table")


# --------- Insert data into tables ---------
def populate_tables(schedule):
    task_table = dynamodb.Table("schedule_task_table")
    subtask_table = dynamodb.Table("schedule_subtask_table")

    for order, task in enumerate(schedule):
        task_id = str(uuid.uuid4())
        has_subtasks = len(task["subtasks"]) > 0

        # Insert into task table
        task_table.put_item(
            Item={
                "task_id": task_id,
                "order": order,
                "task_timing": task["time"],
                "task_name": task["name"],
                "has_subtasks": has_subtasks,
                "residing_column": task["column"],
                "is_completed": False,
            }
        )

        # Insert subtasks
        for sub_order, subtask in enumerate(task["subtasks"]):
            subtask_table.put_item(
                Item={
                    "subtask_id": str(uuid.uuid4()),
                    "task_id": task_id,
                    "order": sub_order,
                    "subtask_name": subtask,
                    "is_completed": False,
                }
            )


# ---------------- MAIN EXECUTION ----------------

# Your schedule data
schedule = [
    {"time": "05:50 - 06:00", "name": "Morning Wakeup", "subtasks": [], "column": 1},
    {"time": "06:00 - 06:30", "name": "Freshen Up", "subtasks": [], "column": 1},
    {
        "time": "06:30 - 06:45",
        "name": "Preworkout Meal",
        "subtasks": ["Eat banana", "Drink preworkout", "Keep Eggs for boiling"],
        "column": 1,
    },
    {"time": "06:45 - 07:00", "name": "Gym Commute", "subtasks": [], "column": 1},
    {"time": "07:00 - 07:10", "name": "Stretching", "subtasks": [], "column": 1},
    {"time": "07:10 - 08:20", "name": "Weight Training", "subtasks": [], "column": 1},
    {"time": "08:20 - 08:40", "name": "Cardio Abs", "subtasks": [], "column": 1},
    {
        "time": "08:40 - 09:00",
        "name": "Home Commute",
        "subtasks": ["Buy Chicken"],
        "column": 1,
    },
    {
        "time": "09:00 - 09:30",
        "name": "Post-Gym Routine",
        "subtasks": ["Room Cleaning", "Drink protein", "Take bath", "Marinate Chicken"],
        "column": 2,
    },
    {
        "time": "09:30 - 10:00",
        "name": "Cooking & Breakfast",
        "subtasks": ["Eggs", "Oats & Paneer"],
        "column": 2,
    },
    {"time": "10:00 - 11:00", "name": "Free Time", "subtasks": [], "column": 2},
    {"time": "11:00 - 12:00", "name": "Office Work", "subtasks": [], "column": 3},
    {"time": "12:00 - 14:00", "name": "Office Work", "subtasks": [], "column": 3},
    {
        "time": "14:00 - 15:30",
        "name": "Cooking & Lunch",
        "subtasks": ["Cook chicken", "Make chapati", "Eat meal"],
        "column": 3,
    },
    {"time": "15:30 - 17:30", "name": "Office Work", "subtasks": [], "column": 3},
    {"time": "17:30 - 18:00", "name": "Walk", "subtasks": [], "column": 4},
    {"time": "18:00 - 19:00", "name": "Work WrapUp", "subtasks": [], "column": 4},
    {"time": "19:00 - 20:00", "name": "Free Time", "subtasks": [], "column": 4},
    {"time": "20:00 - 20:30", "name": "Dinner Prep", "subtasks": [], "column": 4},
    {"time": "20:30 - 21:30", "name": "Dinner Time", "subtasks": [], "column": 5},
    {"time": "21:30 - 22:00", "name": "Wind Down", "subtasks": [], "column": 5},
    {"time": "22:00 - 23:00", "name": "Free Time", "subtasks": [], "column": 5},
    {"time": "23:00 - 23:10", "name": "Sleep Time", "subtasks": [], "column": 5},
]

# Create the tables (run only once)
# create_schedule_task_table()
# create_schedule_subtask_table()

# Populate the tables
populate_tables(schedule)